In [1]:
import numpy as np
import pandas as pd
from json import dump

In [2]:
df = pd.read_csv('ml-100k/u.data', sep = '\t', header = None)
df.columns = ['user_id', 'item_id', 'rating', 'timestamp']

In [3]:
df.head(10)

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [4]:
#количество пользователей (983), фильмов (1682) и рейтингов
users = len(df['user_id'].unique())
films = len(df['item_id'].unique())
ratings = len(df['rating'])

In [5]:
average_user_ratings = float(ratings)/users
average_film_ratings = float(ratings)/films
completeness = float(ratings)/(films*users)

In [6]:
#Готовим таблицу для агрегации всех данных по пользователям
table = pd.DataFrame(df.groupby(['user_id'])['rating'].count())
table.reset_index(inplace = True)
table.columns = ['user_id', 'num_rates']

In [7]:
new_col = pd.DataFrame(df.groupby(['user_id'])['rating'].sum())
new_col.reset_index(inplace = True)
new_col.columns = ['user_id', 'sum_rates']
table['sum_rates'] = new_col['sum_rates']
table['mean_rates'] = table['sum_rates']/table['num_rates']

In [8]:
# Формируем словарь user_dic для каждого пользователя в формате {фильм: рейтинг} по всем оцененным им фильмам
# user_dic[i] - словарь для i+1 пользователя
user_dic = {}
for i in range(1,users+1):
    tmp = pd.DataFrame()
    tmp = df[df['user_id'] == i].copy() # выбираем для каждого пользователя его оценки
    tmp2 = tmp.drop(['user_id', 'timestamp'], axis = 1).set_index('item_id') # удаляем ненужные столбцы и заменяем индекс на номер фильма
    user_dic[i-1] = tmp2.T.to_dict('records') # переводим полученную таблицу в словарь. Вычитаем единицу для приведения i к нулю

In [9]:
# формируем массив из пересечения множеств фильмов, оцененных целевым пользователем с каждым другим пользователем
inters = []
a = 592 # наш целевой пользователь
for i in range(users):
    inters.append(list((set(user_dic[a-1][0].keys())).intersection(set(user_dic[i][0].keys()))))

In [10]:
# на основании пересечения множеств формируем новый словарь new_user_dic для каждого пользователя в который войдут
# общщие с целевым пользователем номера фильмов и их оценка этого фильма
new_user_dic = {}
for j in range(users):
    tmp = {}
    for i in inters[j]:
        tmp[i] = user_dic[j][0][i]
    new_user_dic[j] = tmp

In [11]:
# добавляем в таблицу информацию по кол-во совместно оцененных фильмов и поправочный множитель корреляции Пирсона
new_col2 = []
for i in range(users):
    new_col2.append(len(new_user_dic[i]))
table['num_inter'] = pd.DataFrame(new_col2)
table['pear_correct'] = table['num_inter'].apply(lambda x: min(float(x)/50,1))

In [12]:
# для каждого пользователя находим меру его схожести с целевым пользователем 
from math import sqrt
rat_a = table.iloc[a-1,3] # средний ретинг по всем фильмам заданного пользователя
sim = []
for u in range(users):
    rat_u = table.iloc[u,3] # средний ретинг по всем фильмам текущего пользователя
    sum1, sum21, sum22 = 0,0,0
    for film in inters[u]:
        s1 = new_user_dic[u][film] - rat_u
        s2 = new_user_dic[a-1][film] - rat_a
        sum1 += s1*s2
        sum21 +=s1*s1
        sum22 +=s2*s2
    sim.append(sum1/(sqrt(sum21)*sqrt(sum22))) 

In [13]:
table['pearson'] = pd.DataFrame(sim)
table['pearson_corr'] = table['pear_correct']*table['pearson']

In [14]:
# находим 30 ближайших соседей нашего заданного пользователя
table2 = table.copy()
pearson_neighbours = list(table2.sort_values(by = 'pearson_corr', ascending=False).iloc[1:31,0])

In [56]:
print(pearson_neighbours)

[397, 615, 453, 303, 267, 526, 916, 457, 345, 890, 595, 293, 823, 394, 21, 537, 640, 69, 1, 276, 379, 249, 94, 59, 683, 854, 216, 831, 323, 933]


In [16]:
# рассчитываем предполагаемые рейтинги по всем фильмам. которые не смотрел наш пользователь 
rec_rat = {}
for film in range(1,films+1):
    if film not in user_dic[a-1][0].keys(): #выбираем фильм, который пользователь не оценивал
        sum1, sum2 = 0,0
        for user in pearson_neighbours: # используем для расчетов пользователей из 30-ти ближайших соседей
            if film in user_dic[user-1][0].keys(): # выбираем для текущего пользователя те фильмы, которые он оценивал
                rui = user_dic[user-1][0][film] # его оценка этого фильма 
                ru = table.iloc[user-1,3] # его средний рейтинг
                sau = table.iloc[user-1,7] # мера сходства пользователей с учётом поправочного коэф-та
                sum1 += sau*(rui-ru) #числитель дроби
                sum2 += abs(sau) #знаменатель дроби
        if sum2 != 0:
            rec_rat[film] = table.iloc[a-1,3] + sum1/sum2

In [58]:
[x[0] for x in sorted(rec_rat.items(), key=lambda (k, v): v, reverse=True)[:10]]

[656, 543, 671, 493, 607, 113, 115, 119, 1211, 635]

In [57]:
pearson_top10 = [489, 114, 989, 421, 613, 107, 723, 937, 1053, 1086]

In [19]:
with open("../lab08.json", "w") as outfile:
    dump({'average_film_ratings':average_film_ratings, 'average_user_ratings':average_user_ratings,
         'completeness': completeness, 'pearson_neighbours': pearson_neighbours,
         'pearson_top10': pearson_top10}, outfile, indent=None)

In [20]:
average_rating = float(df['rating'].sum())/df['rating'].count()

In [21]:
# рассчитываем базовый предиктор для каждого пользователя: bu[i] - предиктор для i+1 пользователя
bu = []
for u in range(users):
    s1 = 0
    for film in user_dic[u][0].keys():
        s1 += (user_dic[u][0][film] - average_rating)
    bu.append((s1/(10+len(user_dic[u][0]))))

In [22]:
# Формируем словарь film_dic для каждого фильма в формате {пользователь: рейтинг} по всем пользователям, 
# оценившим данный фильм
# film_dic[i] - словарь для i+1 фильма
film_dic = {}
for i in range(1,films+1):
    tmp = pd.DataFrame()
    tmp = df[df['item_id'] == i].copy()
    tmp2 = tmp.drop(['item_id', 'timestamp'], axis = 1).set_index('user_id')
    film_dic[i-1] = tmp2.T.to_dict('records')

In [23]:
# рассчитываем базовый предиктор для каждого фильма: bi[i] - предиктор для i+1 фильма
bi = []
for i in range(films):
    s1 = 0
    for user in film_dic[i][0].keys():
        s1 += (film_dic[i][0][user] - bu[user-1] - average_rating)
    bi.append((float(s1)/(25+len(film_dic[i][0]))))

In [24]:
# формируем матрицу рейтингов по каждому фильму и пользователю, заполняя пропуски нулями. 
# каждая строка матрицы - вектор рейтингов данного фильма
reiting = np.zeros((films,users))
for i in range(films):
    for j in range(users):
        key_exists = ((j+1) in film_dic[i][0].keys()) # корректируем только те ячейки, по которым есть оценка фильма
        if key_exists:
            reiting[i][j] = film_dic[i][0][j+1] - average_rating - bu[j] - bi[i]

In [25]:
reiting.shape

(1682, 943)

In [26]:
from tqdm import tqdm
from numpy.linalg import norm
# рассчитываем попарные меры близости (косинус) для всех фильмов - перемножаем скалярно вектора и делим на произведение их норм
# в каждой строке полученной матрицы - косинус между фильмом из строки i и столбца j. Матрица получится симметричной относительно главной диагонали
sim_matrix = np.zeros((films,films))
for i in tqdm(range(films)):
    for j in range(films):
        sim_matrix[i][j] = (reiting[i].dot(reiting[j]))/(norm(reiting[i])*norm(reiting[j]))

100%|██████████| 1682/1682 [01:10<00:00, 23.97it/s]


In [27]:
print(sim_matrix[4][5])

-0.0319887084383


In [28]:
from tqdm import tqdm
from numpy.linalg import norm
# рассчитываем абсолютные попарные меры близости (косинус) для всех фильмов - перемножаем скалярно вектора и делим на произведение их норм
# в каждой строке полученной матрицы - косинус между фильмом из строки i и столбца j. Матрица получится симметричной относительно главной диагонали
sim_matrix_abs = np.zeros((films,films))
for i in tqdm(range(films)):
    for j in range(films):
        sim_matrix_abs[i][j] = abs(reiting[i].dot(reiting[j]))/(norm(reiting[i])*norm(reiting[j]))

100%|██████████| 1682/1682 [01:11<00:00, 23.63it/s]


In [29]:
sim_matr = pd.DataFrame(sim_matrix)

In [30]:
sim_matr.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,1.000000,-0.003603,-0.012158,-0.035144,0.039609,0.022780,0.021228,0.052695,-0.004018,-0.015940,...,0.077118,0.0,0.00000,0.00000,0.028896,0.0,0.0,0.0,0.039629,-0.018718
1,-0.003603,1.000000,-0.001984,0.050458,0.002228,-0.003639,-0.016975,0.040146,-0.089526,-0.007177,...,0.000000,0.0,0.00000,0.00000,0.000000,0.0,0.0,0.0,0.022621,0.016415
2,-0.012158,-0.001984,1.000000,-0.113536,-0.005417,0.060713,-0.024408,-0.094746,-0.016745,-0.039964,...,0.000000,0.0,0.00000,0.00000,0.183583,0.0,0.0,0.0,0.000000,0.004880
3,-0.035144,0.050458,-0.113536,1.000000,-0.149507,-0.019089,0.005145,0.110511,0.033087,0.010968,...,0.000000,0.0,-0.11599,-0.11599,0.089827,0.0,0.0,0.0,0.001824,-0.048334
4,0.039609,0.002228,-0.005417,-0.149507,1.000000,-0.031989,-0.015494,0.038203,-0.042447,-0.036512,...,0.000000,0.0,0.00000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.027241


In [31]:
def top_30(matrix, item):
    top30 = matrix.iloc[:,item].sort_values(ascending=False)
    top30 = pd.DataFrame(top30)
    top30.reset_index(inplace = True)
    top30.columns = ['item_id_1', 'sim']
    top30['film_id'] = top30['item_id_1'].apply(lambda x: x+1)
    films_30 = list(top30.iloc[1:31,2])
    sims_30 = list(top30.iloc[1:31,1])
    return films_30, sims_30

In [32]:
top30 = sim_matr.iloc[:,2-1].sort_values(ascending=False)
top30 = pd.DataFrame(top30)
top30.reset_index(inplace = True)
top30.columns = ['item_id', 'sim']
items_30 = list(top30.iloc[1:31,0])
sims_30 = list(top30.iloc[1:31,1])

In [33]:
top30

,item_id,sim
0,1,1.000000
1,384,0.215964
2,565,0.208508
3,1418,0.192721
4,53,0.190520
5,232,0.187075
6,925,0.183361
7,1554,0.177650
8,807,0.177418
9,404,0.175736


In [34]:
items_30_film = [i+1 for i in items_30]

In [35]:
all_items = set(i for i in range(1, films+1))
user_items = set(user_dic[a-1][0].keys())
not_user_items = all_items.difference(user_items)

In [36]:
reit_kr = {}
for film in tqdm(not_user_items):
    items, sims = top_30(sim_matr, film-1)
    si = list(set(items).intersection(set(user_dic[a-1][0].keys())))
    if len(si) != 0:
        sum1, sum2 = 0, 0
        for j in si:
            sum1 += sims[items.index(j)]*(user_dic[a-1][0][j] - (bu[a-1]+bi[j-1]+average_rating))
            sum2 += abs(sims[items.index(j)])
        reit_kr[film] = (bu[a-1] + bi[j-1] + average_rating) + sum1/sum2

100%|██████████| 1322/1322 [00:04<00:00, 264.64it/s]


In [37]:
reit_kr

{2: 4.0010930679642129,
 5: 2.5324186826002837,
 6: 3.5789680919972717,
 10: 4.7632078652328875,
 16: 3.2713816798026087,
 17: 3.821756598848923,
 18: 3.0789222368078697,
 19: 4.1428530200860898,
 21: 2.1915649462970976,
 25: 4.0826435061868445,
 26: 4.2703731229052755,
 27: 0.99999999999999978,
 29: 3.0,
 30: 4.409122150011334,
 31: 4.09285430135614,
 33: 4.3512598234709801,
 35: 2.9967754265557947,
 36: 3.0,
 37: 5.0,
 38: 3.8488950863660021,
 39: 3.9683057512553859,
 40: 1.0,
 41: 5.0,
 43: 5.0,
 44: 5.0,
 45: 4.0,
 46: 5.0,
 49: 5.0,
 51: 3.1782297795723711,
 52: 5.0539800650615732,
 53: 5.1825882503849146,
 54: 3.3755177022517282,
 57: 4.0,
 62: 3.7879392600317363,
 63: 3.3574296498808662,
 65: 4.7650465108915592,
 66: 5.0677389963504282,
 67: 4.0889574515078229,
 68: 3.156170426882166,
 72: 4.7434369330345909,
 73: 3.8991953109153434,
 74: 2.6066807021432474,
 75: 3.6143568147803635,
 76: 4.6197928270083528,
 77: 4.8383281626747543,
 78: 2.4884973797901586,
 80: 4.0,
 82: 4.60718

In [59]:
[ x for x in sorted(reit_kr.items(), key=lambda (k, v): v, reverse=True)[:10]]

[(780, 5.4319508362597393),
 (1603, 5.3999625282792678),
 (177, 5.3046841665253668),
 (1309, 5.2308756008026505),
 (1310, 5.2308756008026505),
 (584, 5.2272599201860537),
 (1482, 5.2124733691530167),
 (693, 5.2001372545537876),
 (53, 5.1825882503849146),
 (1489, 5.1549044314961394)]

In [60]:
predicators_positive_top10 = [780, 1603, 177, 1309, 1310, 584, 1482, 693, 53, 1489]
predicators_top10 = [780, 1603, 177, 1309, 1310, 584, 1482, 693, 53, 1489]

In [61]:
with open("../lab08s.json", "w") as outfile:
    dump({'average_rating': average_rating, 'predicators_positive_top10': predicators_positive_top10,
         'predicators_top10': predicators_top10}, outfile, indent=None)